# Orchestrating Jobs, Model Registration, and Continuous Deployment with Amazon SageMaker in a secure environment

Amazon SageMaker offers Machine Learning application developers and Machine Learning operations engineers the ability to orchestrate SageMaker jobs and author reproducible Machine Learning pipelines, deploy custom-build models for inference in real-time with low latency or offline inferences with Batch Transform, and track lineage of artifacts. You can institute sound operational practices in deploying and monitoring production workflows, deployment of model artifacts, and track artifact lineage through a simple interface, adhering to safety and best-practice paradigmsfor Machine Learning application development.

The SageMaker Pipelines service supports a SageMaker Machine Learning Pipeline Domain Specific Language (DSL), which is a declarative Json specification. This DSL defines a Directed Acyclic Graph (DAG) of pipeline parameters and SageMaker job steps. The SageMaker Python Software Developer Kit (SDK) streamlines the generation of the pipeline DSL using constructs that are already familiar to engineers and scientists alike.

The SageMaker Model Registry is where trained models are stored, versioned, and managed. Data Scientists and Machine Learning Engineers can compare model versions, approve models for deployment, and deploy models from different AWS accounts, all from a single Model Registry. SageMaker enables customers to follow the best practices with ML Ops and getting started right. Customers are able to standup a full ML Ops end-to-end system with a single API call.

## SageMaker Pipelines

Amazon SageMaker Pipelines support the following activites:

* Pipelines - A Directed Acyclic Graph of steps and conditions to orchestrate SageMaker jobs and resource creation.
* Processing Job steps - A simplified, managed experience on SageMaker to run data processing workloads, such as feature engineering, data validation, model evaluation, and model interpretation.
* Training Job steps - An iterative process that teaches a model to make predictions by presenting examples from a training dataset.
* Conditional step execution - Provides conditional execution of branches in a pipeline.
* Registering Models - Creates a model package resource in the Model Registry that can be used to create deployable models in Amazon SageMaker.
* Creating Model steps - Create a model for use in transform steps or later publication as an endpoint.
* Parametrized Pipeline executions - Allows pipeline executions to vary by supplied parameters.
* Transform Job steps - A batch transform to preprocess datasets to remove noise or bias that interferes with training or inference from your dataset, get inferences from large datasets, and run inference when you don't need a persistent endpoint.

## Layout of the SageMaker ModelBuild Project Template

The template provides a starting point for bringing your SageMaker Pipeline development to production.

```
|-- buildspec.yml
|-- CONTRIBUTING.md
|-- dataset
|   `-- abalone-dataset.csv
|-- pipelines
|   |-- abalone
|   |   |-- evaluate.py
|   |   |-- __init__.py
|   |   |-- pipeline.py
|   |   `-- preprocess.py
|   |-- get_pipeline_definition.py
|   |-- __init__.py
|   |-- run_pipeline.py
|   |-- _utils.py
|   `-- __version__.py
|-- README.md
|-- sagemaker-pipelines-project.ipynb
|-- setup.cfg
|-- setup.py
|-- tests
|   `-- test_pipelines.py
`-- tox.ini
```

A description of some of the artifacts is provided below:
<br/><br/>
Your codebuild execution instructions:
```
|-- buildspec.yml
```
<br/><br/>
Your pipeline artifacts, which includes a pipeline module defining the required `get_pipeline` method that returns an instance of a SageMaker pipeline, a preprocessing script that is used in feature engineering, and a model evaluation script to measure the Mean Squared Error of the model that's trained by the pipeline:

```
|-- pipelines
|   |-- abalone
|   |   |-- evaluate.py
|   |   |-- __init__.py
|   |   |-- pipeline.py
|   |   `-- preprocess.py

```
<br/><br/>
Utility modules for getting pipeline definition jsons and running pipelines:

```
|-- pipelines
|   |-- get_pipeline_definition.py
|   |-- __init__.py
|   |-- run_pipeline.py
|   |-- _utils.py
|   `-- __version__.py
```
<br/><br/>
Python package artifacts:
```
|-- setup.cfg
|-- setup.py
```
<br/><br/>
A stubbed testing module for testing your pipeline as you develop:
```
|-- tests
|   `-- test_pipelines.py
```
<br/><br/>
The `tox` testing framework configuration:
```
`-- tox.ini
```

### A SageMaker Pipeline

The pipeline that we create follows a typical Machine Learning Application pattern of pre-processing, training, evaluation, and conditional model registration and publication, if the quality of the model is sufficient.

![A typical ML Application pipeline](img/pipeline-full.png)

### Getting some constants

We get some constants from the local execution environment.

In [28]:
import boto3
import sagemaker
import sagemaker.session
import json

sm = boto3.client("sagemaker")
ssm = boto3.client("ssm")

def get_environment(project_name):
    r = sm.describe_domain(
            DomainId=sm.describe_project(
                ProjectName=project_name
                )["CreatedBy"]["DomainId"]
        )
    del r["ResponseMetadata"]
    del r["CreationTime"]
    del r["LastModifiedTime"]
    r = {**r, **r["DefaultUserSettings"]}
    del r["DefaultUserSettings"]

    i = {
        **r,
        **{t["Key"]:t["Value"] 
            for t in sm.list_tags(ResourceArn=r["DomainArn"])["Tags"] 
            if t["Key"] in ["EnvironmentName", "EnvironmentType"]}
    }

    i["DataBucketName"]=ssm.get_parameter(Name=f"{i['EnvironmentName']}-{i['EnvironmentType']}-data-bucket-name")["Parameter"]["Value"]
    i["ModelBucketName"]=ssm.get_parameter(Name=f"{i['EnvironmentName']}-{i['EnvironmentType']}-model-bucket-name")["Parameter"]["Value"]
    i["S3VPCEId"]=ssm.get_parameter(Name=f"{i['EnvironmentName']}-{i['EnvironmentType']}-s3-vpce-id")["Parameter"]["Value"]

    return i

def get_session(default_bucket):
    """Gets the sagemaker session based on the region.

    Args:
        region: the aws region to start the session
        default_bucket: the bucket to use for storing the artifacts

    Returns:
        sagemaker.session.Session instance
    """

    boto_session = boto3.Session()

    sagemaker_client = boto_session.client("sagemaker")
    runtime_client = boto_session.client("sagemaker-runtime")
    return sagemaker.session.Session(
        boto_session=boto_session,
        sagemaker_client=sagemaker_client,
        sagemaker_runtime_client=runtime_client,
        default_bucket=default_bucket,
    )

Get data science environment data - we need later for a proper network parameters setup:

In [29]:
#·use the current project name
env_data = get_environment(project_name="test22")
print(f"Environment data:\n{json.dumps(env_data, indent=2)}")

Environment data:
{
  "DomainArn": "arn:aws:sagemaker:us-east-1:949335012047:domain/d-nuu0qocmyhd6",
  "DomainId": "d-nuu0qocmyhd6",
  "DomainName": "sm-mlops-dev-us-east-1-sagemaker-domain",
  "HomeEfsFileSystemId": "fs-a3d92317",
  "Status": "InService",
  "AuthMode": "IAM",
  "AppNetworkAccessType": "VpcOnly",
  "SubnetIds": [
    "subnet-0a338babaa635e3e6"
  ],
  "Url": "https://d-nuu0qocmyhd6.studio.us-east-1.sagemaker.aws",
  "VpcId": "vpc-067a7f5454b7649a5",
  "KmsKeyId": "8a9a07e1-bfa0-4141-99d1-5efb68df86fa",
  "ExecutionRole": "arn:aws:iam::949335012047:role/service-role/sm-mlops-env-EnvironmentIAM-SageMakerExecutionRole-184Y5HERXPK2Q",
  "SecurityGroups": [
    "sg-09d0ae98a2abd26f5"
  ],
  "EnvironmentName": "sm-mlops",
  "EnvironmentType": "dev",
  "DataBucketName": "sm-mlops-dev-us-east-1-data",
  "ModelBucketName": "sm-mlops-dev-us-east-1-models",
  "S3VPCEId": "vpce-073c182393ba71594"
}


Setup some constants:

In [30]:
sagemaker_session = get_session(env_data["DataBucketName"])

region = boto3.Session().region_name
role = env_data["ExecutionRole"]
data_bucket = sagemaker_session.default_bucket()
model_bucket = env_data["ModelBucketName"]

print(f"Region: {region}")
print(f"Pipeline execution role: {role}")
print(f"Pipeline data bucket: {data_bucket}")

# Change these to reflect your project/business name or if you want to separate ModelPackageGroup/Pipeline from the rest of your team
model_package_group_name = f"AbaloneModelPackageGroup-Example"
pipeline_name = f"AbalonePipeline-Example"

us-east-1
arn:aws:iam::949335012047:role/service-role/sm-mlops-env-EnvironmentIAM-SageMakerExecutionRole-184Y5HERXPK2Q
sm-mlops-dev-us-east-1-data


### Get the pipeline instance

Here we get the pipeline instance from your pipeline module (`pipelines/abalone/pipeline.py`) so that we can work with it.

In [32]:
from pipelines.abalone.pipeline import get_pipeline


pipeline = get_pipeline(
    region=region,
    role=role,
    data_bucket=data_bucket,
    model_bucket=model_bucket,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name,
)

{'UserId': 'AROA52CGGWLHXAEJ73C27:SageMaker', 'Account': '949335012047', 'Arn': 'arn:aws:sts::949335012047:assumed-role/sm-mlops-env-EnvironmentIAM-SageMakerExecutionRole-184Y5HERXPK2Q/SageMaker', 'ResponseMetadata': {'RequestId': '1fa5d12b-ed0b-4f71-82d9-844b73c22669', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '1fa5d12b-ed0b-4f71-82d9-844b73c22669', 'content-type': 'text/xml', 'content-length': '489', 'date': 'Wed, 21 Apr 2021 14:06:59 GMT'}, 'RetryAttempts': 0}}
Creating the pipeline 'AbalonePipeline-Example': us-east-1/arn:aws:iam::949335012047:role/service-role/sm-mlops-env-EnvironmentIAM-SageMakerExecutionRole-184Y5HERXPK2Q/sm-mlops-dev-us-east-1-data/sm-mlops-dev-us-east-1-models/AbaloneModelPackageGroup-Example
##### Creating sagemaker session, bucket: sm-mlops-dev-us-east-1-data
#### Sagemaker session created: <sagemaker.session.Session object at 0x7fa0f1c0b110>
#### Setting up the pipeline parameters
#### Setting up the parameter: ProcessingInstanceCount
#### 

### Submit the pipeline to SageMaker and start execution

Let's submit our pipeline definition to the workflow service. The role passed in will be used by the workflow service to create all the jobs defined in the steps.

In [34]:
print(f"Use this role to run the pipeline: {role}")
pipeline.upsert(role_arn=role)

Use this role to run the pipeline: arn:aws:iam::949335012047:role/service-role/sm-mlops-env-EnvironmentIAM-SageMakerExecutionRole-184Y5HERXPK2Q


No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


{'PipelineArn': 'arn:aws:sagemaker:us-east-1:949335012047:pipeline/abalonepipeline-example',
 'ResponseMetadata': {'RequestId': '1b4ba18e-0576-4916-a15e-0b93ff1cb151',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1b4ba18e-0576-4916-a15e-0b93ff1cb151',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '91',
   'date': 'Wed, 21 Apr 2021 14:08:30 GMT'},
  'RetryAttempts': 0}}

We'll start the pipeline, accepting all the default parameters.

Values can also be passed into these pipeline parameters on starting of the pipeline, and will be covered later. 

In [35]:
execution = pipeline.start()

### Pipeline Operations: examining and waiting for pipeline execution

Now we describe execution instance and list the steps in the execution to find out more about the execution.

In [36]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:949335012047:pipeline/abalonepipeline-example',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:949335012047:pipeline/abalonepipeline-example/execution/zrnq43zw69ib',
 'PipelineExecutionDisplayName': 'execution-1619014205863',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2021, 4, 21, 14, 10, 5, 776000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 4, 21, 14, 10, 5, 776000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:949335012047:user-profile/d-nuu0qocmyhd6/sm-mlops-dev-us-east-1-user-profile',
  'UserProfileName': 'sm-mlops-dev-us-east-1-user-profile',
  'DomainId': 'd-nuu0qocmyhd6'},
 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:949335012047:user-profile/d-nuu0qocmyhd6/sm-mlops-dev-us-east-1-user-profile',
  'UserProfileName': 'sm-mlops-dev-us-east-1-user-profile',
  'DomainId': 'd-nuu0qocmyhd6'},
 'ResponseMetadata': {'RequestId

We can wait for the execution by invoking `wait()` on the execution:

In [ ]:
execution.wait()

We can list the execution steps to check out the status and artifacts:

In [ ]:
execution.list_steps()

### Parametrized Executions

We can run additional executions of the pipeline specifying different pipeline parameters. The parameters argument is a dictionary whose names are the parameter names, and whose values are the primitive values to use as overrides of the defaults.

Of particular note, based on the performance of the model, we may want to kick off another pipeline execution, but this time on a compute-optimized instance type and set the model approval status automatically be "Approved". This means that the model package version generated by the `RegisterModel` step will automatically be ready for deployment through CI/CD pipelines, such as with SageMaker Projects.

In [ ]:
execution = pipeline.start(
    parameters=dict(
        ProcessingInstanceType="ml.c5.xlarge",
        ModelApprovalStatus="Approved",
    )
)

In [ ]:
execution.wait()

In [ ]:
execution.list_steps()